In [8]:
import pandas as pd
import numpy as np

# Définition des chemins de fichiers
chemin_fichier = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_nutriments_complet.csv"
chemin_sortie = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_scores_sante_normalises.csv"

def normaliser_valeurs_nutritionnelles(row):
    """Normalise les valeurs nutritionnelles pour une portion standard"""
    portions_standard = {
        'Viande': 200,
        'Fast-food': 300,
        'Fusion': 350,
        'Comfort-food': 300,
        'default': 300
    }
    
    portion = portions_standard.get(row['Catégorie'], portions_standard['default'])
    poids_total = sum([
        row['Protéines (g)'],
        row['Lipides (g)'],
        row['Glucides (g)']
    ])
    
    if poids_total == 0:
        return pd.Series({
            'Protéines (g)': row['Protéines (g)'],
            'Lipides (g)': row['Lipides (g)'],
            'Glucides (g)': row['Glucides (g)'],
            'Calories (kcal)': row['Calories (kcal)'],
            'Fibres (g)': row['Fibres (g)'],
            'Sucres (g)': row['Sucres (g)'],
            'Sodium (mg)': row['Sodium (mg)']
        })
    
    facteur = portion / poids_total if poids_total > 0 else 1
    
    # S'assurer que toutes les colonnes sont présentes
    colonnes_requises = ['Protéines (g)', 'Lipides (g)', 'Glucides (g)', 
                        'Calories (kcal)', 'Fibres (g)', 'Sucres (g)', 'Sodium (mg)']
    
    resultat = {}
    for col in colonnes_requises:
        resultat[col] = row[col] * facteur if col in row else 0
    
    return pd.Series(resultat)

try:
    # Lecture du fichier avec gestion d'erreur
    print(f"Tentative de lecture du fichier : {chemin_fichier}")
    df = pd.read_csv(chemin_fichier)
    print("Fichier chargé avec succès!")
    
    # Vérifier les colonnes présentes
    print("\nColonnes présentes dans le fichier :")
    print(df.columns.tolist())

    # S'assurer que toutes les colonnes nécessaires sont présentes
    colonnes_requises = ['Protéines (g)', 'Lipides (g)', 'Glucides (g)', 
                        'Calories (kcal)', 'Fibres (g)', 'Sucres (g)', 'Sodium (mg)', 'Catégorie']
    
    manquantes = [col for col in colonnes_requises if col not in df.columns]
    if manquantes:
        raise ValueError(f"Colonnes manquantes dans le fichier : {manquantes}")

    # Normalisation des valeurs nutritionnelles
    colonnes_nutriments = ['Protéines (g)', 'Lipides (g)', 'Glucides (g)', 
                          'Calories (kcal)', 'Fibres (g)', 'Sucres (g)', 'Sodium (mg)']
    
    # Créer un nouveau DataFrame pour les valeurs normalisées
    df_norm = df.apply(normaliser_valeurs_nutritionnelles, axis=1)
    
    # Mettre à jour uniquement les colonnes de nutriments
    for col in colonnes_nutriments:
        df[col] = df_norm[col]

    df['Score_Sante'] = df.apply(calculer_score_sante, axis=1)
    
    def categoriser_score(score):
        if score >= 8:
            return 'Excellent'
        elif score >= 7:
            return 'Très sain'
        elif score >= 6:
            return 'Sain'
        elif score >= 5:
            return 'Moyen'
        elif score >= 4:
            return 'Peu sain'
        else:
            return 'À éviter'

    df['Categorie_Sante'] = df['Score_Sante'].apply(categoriser_score)

    # Affichage des résultats...
    print("\nAnalyse nutritionnelle des plats (valeurs normalisées par portion) :")
    print("\nTop 5 des plats les plus sains :")
    print(df.sort_values('Score_Sante', ascending=False)[['Plat', 'Score_Sante', 'Categorie_Sante']].head())

    print("\nTop 5 des plats les moins sains :")
    print(df.sort_values('Score_Sante', ascending=True)[['Plat', 'Score_Sante', 'Categorie_Sante']].head())

    print("\nScore moyen par pays :")
    print(df.groupby('Pays')['Score_Sante'].mean().sort_values(ascending=False))

    df.to_csv(chemin_sortie, index=False)
    print(f"\nRésultats sauvegardés dans : {chemin_sortie}")

except FileNotFoundError:
    print(f"Erreur : Le fichier n'a pas été trouvé à l'emplacement : {chemin_fichier}")
except Exception as e:
    print(f"Une erreur est survenue : {str(e)}")





Tentative de lecture du fichier : /Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_nutriments_complet.csv
Fichier chargé avec succès!

Colonnes présentes dans le fichier :
['Plat', 'Pays', 'Catégorie', 'Protéines (g)', 'Lipides (g)', 'Glucides (g)', 'Calories (kcal)', 'Fibres (g)', 'Sucres (g)', 'Sodium (mg)']

Analyse nutritionnelle des plats (valeurs normalisées par portion) :

Top 5 des plats les plus sains :
                 Plat  Score_Sante Categorie_Sante
75     Mac and Cheese          6.9            Sain
41           Gazpacho          6.6            Sain
4   Boeuf bourguignon          5.9           Moyen
53           Saganaki          5.7           Moyen
1               Pizza          5.7           Moyen

Top 5 des plats les moins sains :
               Plat  Score_Sante Categorie_Sante
80    Buffalo Wings          0.6        À éviter
0          Couscous          0.8        À éviter
2      Steak-frites          1.5     